<a href="https://colab.research.google.com/github/HeorhiiY/cv-course/blob/main/project/Project0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [22]:
# !ls "/content/drive/MyDrive/Colab Notebooks/project"


data  train_images  train_images.zip  train_labels.zip	unzipped_files	val_images  val_images.zip


In [4]:
import json

# Define the file path
# geojson_file_path = '/content/drive/MyDrive/Colab Notebooks/project/data/xView_train.geojson'
# local
geojson_file_path = 'data/xView_train.geojson'

# Load and parse the GeoJSON file
with open(geojson_file_path, 'r') as file:
    geojson_data = json.load(file)

# Print the top-level keys
print("Top-level keys in the GeoJSON data:", geojson_data.keys())

# Example: Inspect the 'features' key if it exists
if 'features' in geojson_data:
    print("First feature in the GeoJSON file:")
    print(json.dumps(geojson_data['features'][0], indent=4))  # Pretty-print the first feature

Top-level keys in the GeoJSON data: dict_keys(['crs', 'type', 'features', 'name'])
First feature in the GeoJSON file:
{
    "type": "Feature",
    "properties": {
        "bounds_imcoords": "2712,1145,2746,1177",
        "edited_by": "wwoscarbecerril",
        "cat_id": "1040010028371A00",
        "type_id": 73,
        "ingest_time": "2017/07/24 12:49:09.118+00",
        "index_right": 2356,
        "image_id": "2355.tif",
        "point_geom": "0101000020E6100000616E4E6406A256C03BE6ADA0D6212D40",
        "feature_id": 374410,
        "grid_file": "Grid2.shp"
    },
    "geometry": {
        "type": "Polygon",
        "coordinates": [
            [
                [
                    -90.53169885094464,
                    14.56603647302396
                ],
                [
                    -90.53169885094464,
                    14.56614473506768
                ],
                [
                    -90.53158140073565,
                    14.56614473506768
                

In [2]:
from ultralytics.data.utils import autosplit
from ultralytics.utils.ops import xyxy2xywhn
import json
import os
from pathlib import Path

import numpy as np
from PIL import Image
from tqdm import tqdm

def convert_labels(fname=Path('xView/xView_train.geojson')):
    # Convert xView geoJSON labels to YOLO format
    path = fname.parent
    with open(fname) as f:
        print(f'Loading {fname}...')
        data = json.load(f)

    # Make dirs
    labels = Path(path / 'labels' / 'train')
    os.system(f'rm -rf {labels}')
    labels.mkdir(parents=True, exist_ok=True)

    # xView classes 11-94 to 0-59
    xview_class2index = [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 1, 2, -1, 3, -1, 4, 5, 6, 7, 8, -1, 9, 10, 11,
                          12, 13, 14, 15, -1, -1, 16, 17, 18, 19, 20, 21, 22, -1, 23, 24, 25, -1, 26, 27, -1, 28, -1,
                          29, 30, 31, 32, 33, 34, 35, 36, 37, -1, 38, 39, 40, 41, 42, 43, 44, 45, -1, -1, -1, -1, 46,
                          47, 48, 49, -1, 50, 51, -1, 52, -1, -1, -1, 53, 54, -1, 55, -1, -1, 56, -1, 57, -1, 58, 59]

    shapes = {}
    for feature in tqdm(data['features'], desc=f'Converting {fname}'):
        p = feature['properties']
        if p['bounds_imcoords']:
            id = p['image_id']
            file = path / 'train_images' / id
            if file.exists():  # 1395.tif missing
                try:
                    box = np.array([int(num) for num in p['bounds_imcoords'].split(",")])
                    assert box.shape[0] == 4, f'incorrect box shape {box.shape[0]}'
                    cls = p['type_id']
                    cls = xview_class2index[int(cls)]  # xView class to 0-60
                    assert 59 >= cls >= 0, f'incorrect class index {cls}'

                    # Write YOLO label
                    if id not in shapes:
                        shapes[id] = Image.open(file).size
                    box = xyxy2xywhn(box[None].astype(np.float64), w=shapes[id][0], h=shapes[id][1], clip=True)
                    with open((labels / id).with_suffix('.txt'), 'a') as f:
                        f.write(f"{cls} {' '.join(f'{x:.6f}' for x in box[0])}\n")  # write label.txt
                except Exception as e:
                    print(f'WARNING: skipping one label for {file}: {e}')

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\streb\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
convert_labels(Path(geojson_file_path))

Loading data\xView_train.geojson...


Converting data\xView_train.geojson:   4%|█▍                                  | 23502/601937 [00:12<03:12, 3007.56it/s]

Converting data\xView_train.geojson:   5%|█▋                                  | 27302/601937 [00:13<03:03, 3126.35it/s]

Converting data\xView_train.geojson:   5%|█▊                                  | 30559/601937 [00:15<03:12, 2967.51it/s]

Converting data\xView_train.geojson:   7%|██▌                                 | 41874/601937 [00:18<02:47, 3342.65it/s]

...

Converting data\xView_train.geojson:  83%|█████████████████████████████      | 499903/601937 [02:55<00:33, 3019.34it/s]

Converting data\xView_train.geojson:  98%|██████████████████████████████████▏| 588339/601937 [03:25<00:04, 2834.74it/s]

Converting data\xView_train.geojson:  98%|██████████████████████████████████▎| 589196/601937 [03:25<00:04, 2688.58it/s]

Converting data\xView_train.geojson:  98%|██████████████████████████████████▍| 591431/601937 [03:26<00:03, 2933.02it/s]

Converting data\xView_train.geojson: 100%|███████████████████████████████████| 601937/601937 [03:30<00:00, 2854.79it/s]
